In [17]:
import csv
import numpy as np
import networkx as nx
import math
from scipy.stats import mode
import function
import pickle

# 得到每个小时每个点的进出热度

### 读取node_neis

In [6]:
# 读取node_neis
i = 0
node_neis = {}
csv_reader = csv.reader(open('./data/temp/neighbour.csv', encoding='utf-8'))
for row in csv_reader:
    i+=1
    if i == 1:
        continue
    node_neis[row[0]] = eval(row[1])

### 读取node_region

In [7]:
# 读取node_region
i = 0
node_region = {}
csv_reader = csv.reader(open('./data/temp/region.csv', encoding='utf-8'))
for row in csv_reader:
    i+=1
    if i == 1:
        continue
    node_region[row[0]] = eval(row[1])

## 得到每个小时每个点的出发和到达次数

In [8]:
node_count_hour_start = {}
node_count_hour_end = {}
i = 0
csv_reader = csv.reader(open('./data/train_hard.csv', encoding='utf-8'))
for row in csv_reader:
    i+=1
    if i == 1:
        continue

    start = row[5]
    end = row[6]
    hour = row[7]
    
    if hour not in node_count_hour_start:
        node_count_hour_start[hour] = {}
        node_count_hour_start[hour][start] = 1 
    else:
        if start not in node_count_hour_start[hour]:
            node_count_hour_start[hour][start] = 1 
        else:
            node_count_hour_start[hour][start] += 1 
    
    if hour not in node_count_hour_end:
        node_count_hour_end[hour] = {}
        node_count_hour_end[hour][end] = 1
    else:
        if end not in node_count_hour_end[hour]:
            node_count_hour_end[hour][end] = 1
        else:
            node_count_hour_end[hour][end] += 1

## 得到geofly之前的进、出、总热度



In [9]:
def get_hour_ratio(node_region,node_count_start,node_count_end):
    
    node_region_start_ratio = {}
    node_region_end_ratio = {}
    node_region_overall_ratio = {}
    for node in node_region:#对于所有

        neis = node_region[node]

        total_start = 0
        total_end = 0

        total_node = 0
        total_region = 0

        for nei in neis:
            if nei in node_count_start:
                total_start+=node_count_start[nei]#区域内总出发
                total_region+=node_count_start[nei]#区域内总进出
            if nei in node_count_end:
                total_end+=node_count_end[nei]#区域内总进
                total_region+=node_count_end[nei]#区域内总进出

        if node in node_count_start:#如果该点出发过
            node_region_start_ratio[node] = node_count_start[node]/total_start#region出发热度
            total_node+=node_count_start[node]#计算总热度
        else:
            node_region_start_ratio[node] = -1#不存在

        if node in node_count_end:#如果该点被到达过
            node_region_end_ratio[node] = node_count_end[node]/total_end#region到达热度
            total_node+=node_count_end[node]#计算总热
        else:
            node_region_end_ratio[node] = -1#不存在

        if total_region == 0 or total_node == 0:
            node_region_overall_ratio[node] = -1
        else:
            node_region_overall_ratio[node] = total_node/total_region
            
    return node_region_start_ratio,node_region_end_ratio,node_region_overall_ratio


In [14]:
node_region_hour_start_ratio = {}
node_region_hour_end_ratio = {}
node_region_hour_overall_ratio = {}
for hour in node_count_hour_start:
    node_region_hour_start_ratio[hour],\
    node_region_hour_end_ratio[hour],\
    node_region_hour_overall_ratio[hour] = get_hour_ratio(node_region,\
                                                          node_count_hour_start[hour],node_count_hour_end[hour])

In [34]:
with open('./data/temp/feature_table/node_region_hour_start_ratio','wb+') as f:
    pickle.dump(node_region_hour_start_ratio, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/feature_table/node_region_hour_end_ratio','wb+') as f:
    pickle.dump(node_region_hour_end_ratio, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/feature_table/node_region_hour_overall_ratio','wb+') as f:
    pickle.dump(node_region_hour_overall_ratio, f, pickle.HIGHEST_PROTOCOL)

# geofly后

## 得到每个小时每个点的出发和到达次数(用于计算热度比例)

In [35]:
node_count_hour_start = {}
node_count_hour_end = {}
i = 0
csv_reader = csv.reader(open('./data/train_hard.csv', encoding='utf-8'))
for row in csv_reader:
    i+=1
    if i == 1:
        continue

    start = row[5]
    end = row[6]
    hour = row[7]
    
    if hour not in node_count_hour_start:
        node_count_hour_start[hour] = {}
        node_count_hour_start[hour][start] = 1 
    else:
        if start not in node_count_hour_start[hour]:
            node_count_hour_start[hour][start] = 1 
        else:
            node_count_hour_start[hour][start] += 1 
    
    if hour not in node_count_hour_end:
        node_count_hour_end[hour] = {}
        node_count_hour_end[hour][end] = 1
    else:
        if end not in node_count_hour_end[hour]:
            node_count_hour_end[hour][end] = 1
        else:
            node_count_hour_end[hour][end] += 1

## 得到geofly每个小时每个点的出发和到达次数

In [36]:
node_count_hour_start_geofly = {}
node_count_hour_end_geofly = {}
i = 0
csv_reader = csv.reader(open('./data/train_start_geofly.csv', encoding='utf-8'))
for row in csv_reader:
    i+=1
    if i == 1:
        continue

    start = row[5]
    end = row[6]
    hour = row[7]
    
    if hour not in node_count_hour_start_geofly:
        node_count_hour_start_geofly[hour] = {}
        node_count_hour_start_geofly[hour][start] = 1 
    else:
        if start not in node_count_hour_start_geofly[hour]:
            node_count_hour_start_geofly[hour][start] = 1 
        else:
            node_count_hour_start_geofly[hour][start] += 1 
            
i = 0
csv_reader = csv.reader(open('./data/train_end_geofly.csv', encoding='utf-8'))
for row in csv_reader:
    i+=1
    if i == 1:
        continue

    start = row[5]
    end = row[6]
    hour = row[7]
    
    if hour not in node_count_hour_end_geofly:
        node_count_hour_end_geofly[hour] = {}
        node_count_hour_end_geofly[hour][end] = 1
    else:
        if end not in node_count_hour_end_geofly[hour]:
            node_count_hour_end_geofly[hour][end] = 1
        else:
            node_count_hour_end_geofly[hour][end] += 1

## 得到每小时geofly的进、出、总热度

In [37]:
def get_hour_ratio_geofly(node_region,node_count_start,node_count_end,node_count_start_geofly,node_count_end_geofly):
    
    node_region_start_ratio = {}
    node_region_end_ratio = {}
    node_region_overall_ratio = {}
    for node in node_region:#对于所有

        neis = node_region[node]

        total_start = 0
        total_end = 0

        total_node = 0
        total_region = 0

        for nei in neis:
            if nei in node_count_start: 
                total_start+=node_count_start[nei]#区域内总出发  只能算没有fly前的次数
                total_region+=node_count_start[nei]#区域内总进出
            if nei in node_count_end:
                total_end+=node_count_end[nei]#区域内总进   只能算没有fly前的次数
                total_region+=node_count_end[nei]#区域内总进出

        if node in node_count_start_geofly:#如果该点出发过
            node_region_start_ratio[node] = node_count_start_geofly[node]/total_start#region出发热度
            total_node+=node_count_start_geofly[node]#计算总热度
        else:
            node_region_start_ratio[node] = -1#不存在

        if node in node_count_end_geofly:#如果该点被到达过
            node_region_end_ratio[node] = node_count_end_geofly[node]/total_end#region到达热度
            total_node+=node_count_end_geofly[node]#计算总热
        else:
            node_region_end_ratio[node] = -1#不存在

        if total_region == 0 or total_node == 0:
            node_region_overall_ratio[node] = -1
        else:
            node_region_overall_ratio[node] = total_node/total_region
            
    return node_region_start_ratio,node_region_end_ratio,node_region_overall_ratio



In [38]:
node_region_hour_start_geofly_ratio = {}
node_region_hour_end_geofly_ratio = {}
node_region_hour_overall_geofly_ratio = {}
for hour in node_count_hour_start_geofly:
    node_region_hour_start_geofly_ratio[hour],\
    node_region_hour_end_geofly_ratio[hour],\
    node_region_hour_overall_geofly_ratio[hour] = get_hour_ratio_geofly(node_region,\
                                                            node_count_hour_start[hour],node_count_hour_end[hour],\
                                                          node_count_hour_start_geofly[hour],\
                                                                 node_count_hour_end_geofly[hour])

In [39]:
with open('./data/temp/feature_table/node_region_hour_start_geofly_ratio','wb+') as f:
    pickle.dump(node_region_hour_start_geofly_ratio, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/feature_table/node_region_hour_end_geofly_ratio','wb+') as f:
    pickle.dump(node_region_hour_end_geofly_ratio, f, pickle.HIGHEST_PROTOCOL)
    
with open('./data/temp/feature_table/node_region_hour_overall_geofly_ratio','wb+') as f:
    pickle.dump(node_region_hour_overall_geofly_ratio, f, pickle.HIGHEST_PROTOCOL)